# XGBoost
* TODO: add averages for cells and stations, NB inference code will need to be adapted too
* TODO: target manipulations/engineering
    * rolling autocorrelation
* TODO: (vector leaf) multi-output regression
* TODO: maybe for week 10-11, we should have radically different approach that does not rely on lag feats, since these will be heavily inpacted by compound errors
    * maybe we should train lin reg for trend and xgboost for seasonality only on idx feats

## Roadmap Note
Regarding your plan to expand the script to first fit a linear model and then apply XGBoost on the residuals, that's a solid approach known as model stacking or residual modeling. This can be set up as a parameter in W&B for flexibility. When you're ready to implement it, you might consider:

* Implementing a Pipeline: Use scikit-learn's Pipeline to chain the linear model and XGBoost.
* Parameterization: Add a parameter in your config (e.g., use_linear_model) to toggle this behavior.
* Logging: Use W&B to track both models' performances separately and combined.

In [1]:
# %%
from tqdm.notebook import tqdm
from wandb.integration.xgboost import WandbCallback
import wandb
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from optuna_integration.xgboost import XGBoostPruningCallback
from optuna_integration.wandb import WeightsAndBiasesCallback
import optuna
import xgboost as xgb
import numpy as np
import polars as pl
import pickle
from typing import Callable
import math
from pathlib import Path
import os
import yaml

import utils

# Manually set the notebook name
os.environ["WANDB_NOTEBOOK_NAME"] = "xgboost_train.ipynb"

In [2]:
# %%
DEBUG = False
config_file_path = Path('configs') / 'autoregressive_config.yaml'

In [3]:
# Load the YAML configuration file
with open(config_file_path, 'r') as file:
    config = yaml.safe_load(file)

# Extract xgb_hyperparams from config
xgb_hyperparams = config.get('xgb_hyperparams', {})

# Merge xgb_hyperparams into config
config.update(xgb_hyperparams)

In [4]:
# %%
# Read the CSV files
data_dir = Path('input-data')
target_dataframes = {
    # This is the target variable
    'thp_vol': pl.read_csv(data_dir / 'traffic_DLThpVol.csv'),
    'prb': pl.read_csv(data_dir / 'traffic_DLPRB.csv'),
    'thp_time': pl.read_csv(data_dir / 'traffic_DLThpTime.csv'),
    'mr_number': pl.read_csv(data_dir / 'traffic_MR_number.csv')
}

# Filter target dataframes based on config
target_dataframes = {
    k: v for k, v in target_dataframes.items() if k in config['target_df_names']}

idx_hour_series = target_dataframes['thp_vol']['']

# Drop the first column (idx hour) from each dataframe
for k in target_dataframes:
    target_dataframes[k] = target_dataframes[k].drop('')

# Debug mode: shorten dataframes and config lists
if DEBUG:
    target_dataframes = {k: v.head(200).select(
        v.columns[:800]) for k, v in target_dataframes.items()}
    config = {k: v[:3] if isinstance(
        v, list) else v for k, v in config.items()}

# Merge xgb_hyperparams into config
config.update(xgb_hyperparams)

In [5]:
# %%
# Initialize W&B
run = wandb.init(
    project="traffic-forecasting-challenge",
    job_type='train',
    entity="esedx12",
    config=config,
    save_code=True,
    mode=('dryrun' if DEBUG else 'online')
)

# Save utils.py to W&B
utils_path = Path('utils.py')
if utils_path.exists():
    wandb.save(str(utils_path))

wandb: Currently logged in as: esedx12. Use `wandb login --relogin` to force relogin


 ## Feature Engineering

 The feature engineering steps are handled by utility functions.

In [6]:
# %%
# Use first config.train_percentage of dataframe rows for training, and the rest for validation and testing
num_rows = len(target_dataframes['thp_vol'])
num_train_rows = round(num_rows * wandb.config.train_percentage)
num_val_rows = round(num_rows * wandb.config.val_percentage)

train_dataframes = {k: v.head(num_train_rows)
                    for k, v in target_dataframes.items()}
train_idx_hour_series = idx_hour_series.head(num_train_rows)

val_dataframes = {k: v.slice(num_train_rows + 1, num_val_rows)
                  for k, v in target_dataframes.items()}
val_idx_hour_series = idx_hour_series.slice(num_train_rows + 1, num_val_rows)

In [14]:

# Create long format dataframes using utility functions
long_train_df = utils.create_long_format_df(
    train_dataframes, train_idx_hour_series, wandb.config)
long_val_df = utils.create_long_format_df(
    val_dataframes, val_idx_hour_series, wandb.config)

target_cols = list(target_dataframes.keys())

# Assuming long_train_df and long_val_df are pandas DataFrames
X_train = long_train_df.drop(columns=target_cols)
y_train = long_train_df[target_cols]

X_val = long_val_df.drop(columns=target_cols)
y_val = long_val_df[target_cols]

wandb.config.update({
    'num_train_samples': len(X_train),
    'num_val_samples': len(X_val),
    'features': X_train.columns.to_list(),
    'targets': y_train.columns.to_list()
})

Error: You must call wandb.init() before wandb.config['use_beam_id_feat']

 ## Train Models
*  TODO if indicated for performance reasons, get the max idx_hour with a null and return it so we can shorten the df for multi-step predict
* TODO also add target transformations (maybe sklearn can help)
* TODO normalize somehow if data is on very different scales for different beams

### Fit models

In [9]:
# %%
models = {}
for target_name in y_train.columns:
    model = xgb.XGBRegressor(
        **xgb_hyperparams, callbacks=[WandbCallback(log_model=True)])
    print(f"\nFitting model for {target_name}:")
    model.fit(
        X_train,
        y_train[target_name],
        eval_set=[(X_train, y_train[target_name]),
                  (X_val, y_val[target_name])],
        verbose=25
    )
    models[target_name] = model


Fitting model for thp_vol:
[0]	validation_0-mae:0.36075	validation_1-mae:0.34949
[19]	validation_0-mae:0.21370	validation_1-mae:0.20862

Fitting model for mr_number:
[0]	validation_0-mae:0.58309	validation_1-mae:0.59078
[19]	validation_0-mae:0.21326	validation_1-mae:0.21630


### Save models

In [10]:
# %%
for target_name, model in models.items():
    model_dir = Path('checkpoints') / wandb.run.name
    model_dir.mkdir(parents=True, exist_ok=True)
    model_path = model_dir / f'{target_name}.ubj'
    pickle.dump(model, open(model_path, 'wb'))
    wandb.save(str(model_path))

 ## Evaluation and Logging

In [11]:
# %%
# Iterate through each model in models
for target, target_model in models.items():
    print(f"Processing target: {target}")

    # Predict
    train_preds = target_model.predict(X_train)
    val_preds = target_model.predict(X_val)

    # Compute MAE values
    train_mae = mean_absolute_error(y_train[target], train_preds)
    val_mae = mean_absolute_error(y_val[target], val_preds)

    # Log the best score to wandb
    # XGBoost does not have best_iteration attribute in scikit-learn API
    evals_result = target_model.evals_result()
    best_iteration = len(evals_result['validation_0']['mae'])  # Last iteration
    best_val_mae = evals_result['validation_1']['mae'][-1]
    best_train_mae = evals_result['validation_0']['mae'][-1]

    wandb.log({
        f'{target}_best_val_mae': best_val_mae,
        f'{target}_best_round': best_iteration,
        f'{target}_best_train_mae': best_train_mae
    })

    # Convert evaluation results to a DataFrame
    eval_df = pl.DataFrame({
        'Round': list(range(1, len(evals_result['validation_0']['mae']) + 1)),
        'Train MAE': evals_result['validation_0']['mae'],
        'Val MAE': evals_result['validation_1']['mae']
    })

    # Log eval_df to wandb
    wandb.log({f'{target}_eval_df': wandb.Table(data=eval_df.to_pandas())})

    # Plot the results using Plotly
    fig = px.line(
        eval_df.to_pandas(),
        x='Round',
        y=['Train MAE', 'Val MAE'],
        labels={'value': 'Mean Absolute Error'},
        title=f'Training and Validation MAE over Boosting Rounds for {target}'
    )

    fig.update_layout(
        legend=dict(
            title='Dataset',
            itemsizing='constant'
        )
    )

    # Log the plot to wandb
    wandb.log({f"{target}_MAE_Plot": fig})

    # Optionally, display the plot
    fig.show()

    print(f"Best Val MAE for {target}: {best_val_mae}")
    print(f"Round: {best_iteration}")

Processing target: thp_vol


Best Val MAE for thp_vol: 0.20862414911785193
Round: 20
Processing target: mr_number


Best Val MAE for mr_number: 0.2163008547167498
Round: 20


In [12]:
# %%
if wandb.config.run_shap:
    # Create a SHAP explainer for the XGBoost model
    # Assuming 'thp_vol' is one of the targets
    target_name = 'thp_vol'
    explainer = shap.TreeExplainer(models[target_name], X_val)

    # Calculate SHAP values for the val set
    shap_values = explainer.shap_values(X_val)

    # Log SHAP plots to wandb
    shap_bar = shap.summary_plot(
        shap_values, X_val, plot_type="bar", show=False)
    plt.savefig("shap_bar.png")
    wandb.log({"SHAP Bar Plot": wandb.Image("shap_bar.png")})
    plt.clf()

    shap_summary = shap.summary_plot(
        shap_values, X_val, show=False)
    plt.savefig("shap_summary.png")
    wandb.log({"SHAP Summary Plot": wandb.Image("shap_summary.png")})
    plt.clf()

    # Optionally, add more SHAP plots as needed

In [13]:
# %%
wandb.finish()

best_iteration,▁▁
best_score,▁█
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
mr_number_best_round,▁
mr_number_best_train_mae,▁
mr_number_best_val_mae,▁
thp_vol_best_round,▁
thp_vol_best_train_mae,▁
thp_vol_best_val_mae,▁
validation_0-mae,▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁█▇▆▅▅▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁
validation_1-mae,▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁█▇▆▆▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁
